In [1]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api

from Linguistic_Features import*
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from textblob import TextBlob

import pandas as pd
import numpy as np
import pickle

unable to import 'smart_open.gcs', disabling that module


In [8]:
def Get_AvgSentence_Embedding(Sentence):
    
    sent_words = word_tokenize(Sentence)
    
    word_embeddings = [] 
    for sent_word in sent_words:
        
        if not sent_word.isalpha():
            continue
        
        if sent_word in vocabulary:
            word_embedding = vectors[sent_word]
            word_embeddings.append(word_embedding)
    
    word_embeddings = np.array(word_embeddings)
    sent_embedding = np.mean(word_embeddings, axis=0)
    
    return sent_embedding

def Load_Trained_Model(ModelPath):
    
    with open(ModelPath, "rb") as file:
        Model_Obj = pickle.load(file)
        
    return Model_Obj

def Task1_Predict(Sentence, Test_Features):
    
    ## Our trained model prediction
    model_predictions = Task1_Model.predict(Test_Features)
    if model_predictions[0] < 0.5:
        model_prediction = 0
    else:
        model_prediction = 1
    
    ## Textblob predictions
    Textblob_prediction = int(TextBlob(Sentence).sentiment.subjectivity)
    
    ## Average Predictions
    Avg_Score = (model_prediction + Textblob_prediction)/2
    
    if Avg_Score < 0.5:
        Avg_Pred = 0
    else:
        Avg_Pred = 1
    
    return Avg_Pred

def Task2_Predict(Sent_Linguistic_Features):
    
    Test_Features = Sent_Linguistic_Features[FeatureList]
    sent_words = list(Sent_Linguistic_Features['word'])
    
    ## Our trained model prediction
    
    ## Naive Bayes
    model_probabilities = Task2_Model1.predict_proba(Test_Features)
    model1_predictions = [(model_proba[0]+model_proba[1])/2 for model_proba in model_probabilities]

    
    ## Random Forest
    model2_predictions = Task2_Model2.predict(Test_Features)
    
    ## Textblob predictions
    Textblob_predictions = []
    for sent_word in sent_words:
        Textblob_predictions.append(TextBlob(sent_word).sentiment.subjectivity)
    
    ## Average Predictions
    Avg_Predictions = []
    
    for i in range(len(sent_words)):
        
        Avg_Score = (model1_predictions[i] + model1_predictions[i] + Textblob_predictions[i])/3
        Pred_Entry = (sent_words[i], Avg_Score)
        Avg_Predictions.append(Pred_Entry)
    
    return Avg_Predictions



In [3]:
def Initialization(Task1Model_Path, Task2Model1_Path, Task2Model2_Path):
    
    global Task1_Model, Task2_Model1, Task2_Model2
    
    Task1_Model = Load_Trained_Model(Task1Model_Path)
    Task2_Model1 = Load_Trained_Model(Task2Model1_Path)
    Task2_Model2 = Load_Trained_Model(Task2Model2_Path)

def Bias_Detection_Suggestions(Sentences, N_Similar=0, TopN_Suggestion=0):
    
    Classification_labels = []
    
    for Sentence in Sentences:
        
        print("\n\n")
        print(Sentence)
    
        ## Step 1: Get average word embedding of the sentence
        Sent_Embedding = Get_AvgSentence_Embedding(Sentence)

        ## Step 2: Use best performing task 1 model to predict id sentence is subjective
        Sent_Embedding_Features = pd.DataFrame([Sent_Embedding])
        task1_Prediction = Task1_Predict(Sentence, Sent_Embedding_Features)
        
        ## Step 3: If sentence is objective: Do nothing
        if task1_Prediction==0:
            Classification_label = 0
            print("Subjectivity not identified in the sentence")
        
        ## Step 3: If sentence is subjective: Identify words inducing subjectivity in the sentence
        else:
            Classification_label = 1
            
            sent_words = word_tokenize(Sentence)
            sent_words = [w for w in sent_words if w.isalpha()]
            
            ## Step 3.1: Get liguistic features of the words in the sentence
            Sent_Linguistic_Features = Get_Sent_Linguistic_Features(Sentence)
            
            ## Ignore stopwords
            Sent_Linguistic_Features = Sent_Linguistic_Features[~Sent_Linguistic_Features['word'].isin(stop_words)]
            
            ## Ignore proper nouns (13-->NNP)
            Sent_Linguistic_Features = Sent_Linguistic_Features[Sent_Linguistic_Features['POS']!=13]
            
            ## Step 3.2: Get subjectivity score for all words in the sentence
            Word_Subjectivity_Scores = Task2_Predict(Sent_Linguistic_Features)
            
            ## Step 3.3: Identify word introducing subjectivity in the sentence (max score)
            max_score = 0
            subjective_word = ""
            
            for i in range(len(Word_Subjectivity_Scores)):
                
                current_word = Word_Subjectivity_Scores[i][0]
                subj_score = Word_Subjectivity_Scores[i][1]
                
                if subj_score > max_score and current_word in vocabulary:
                    max_score = subj_score
                    subjective_word = current_word
                    
            if max_score == 0:
                Classification_labels.append(0)
                print("Subjectivity not identified in the sentence")
                continue
                
            print("Sentence is subjective")
            print("Word inducing subjectivity in the sentence: ", subjective_word)
                
            ## Step 3.4: Get alternative words for identified subjective word 
            
            ## Top 10 : words with similar meaning --> words with similar embedding 
            ##          Words most similar to subjective word in order of decresing similarity
            if N_Similar==0:
                N_Similar = 10
            Similar_words = vectors.most_similar(subjective_word, topn=N_Similar)
            Similar_words.sort(key = lambda x: x[1], reverse=True)
            
            ## Get least subjective top 3 similar words
    
            Similar_words_features = pd.DataFrame(columns=FeatureList)
            Similar_words_str = []
            
            for similar_word in Similar_words:
                
                word_features = Get_Sent_Linguistic_Features(similar_word[0])
                Similar_words_features = pd.concat([Similar_words_features, word_features])
             
            Subjectivity_scores = Task2_Predict(Similar_words_features)
            Subjectivity_scores.sort(key = lambda x: x[1])
            
            Subjective_word_lemma = lemmatizer.lemmatize(subjective_word)
            Filter_words = [tup for tup in Subjectivity_scores if Subjective_word_lemma not in lemmatizer.lemmatize(tup[0]) and tup[1] < max_score]

            if TopN_Suggestion == 0 or TopN_Suggestion>len(Filter_words):
                TopN_Suggestion = len(Filter_words)
            
            TopN_Suggested_Words = Filter_words[:TopN_Suggestion]
            
            print("Alternate word suggetions: ")
            for suggestion in TopN_Suggested_Words : 
                print(suggestion[0])
            
         
        Classification_labels.append(Classification_label)
    return Classification_labels
        

In [6]:
## Initialization 

Folder_Path = "/Users/pranjali/Downloads/Wiki_BiasDetection"

Vocab_Filepath = Folder_Path + "/Data/Embeddings/pretrained_modified.txt"
Task1Model_Path = Folder_Path + "/Saved_Models/NN/NN_Task1_Trained_Model.pkl"
Task2Model1_Path = Folder_Path + "/Saved_Models/NB/NB_Task2_Trained_Model.pkl"
Task2Model2_Path = Folder_Path + "/Saved_Models/RF/RF_Task2_Trained_Model.pkl"

vectors = KeyedVectors.load_word2vec_format(Vocab_Filepath, binary=False)
vocabulary = vectors.wv.vocab

stop_words = set(stopwords.words('english')) 

lemmatizer = WordNetLemmatizer()

FeatureList = ["POS", "POS_Prev", "POS_Next", "Sent_Position", "Hedge", "Hedge_Context", 
               "Factive", "Factive_Context", "Assertive", "Assertive_Context",
               "Implicative", "Implicative_Context", "Report", "Report_Context", 
                "Entailment", "Entailment_Context", "StrongSub", "StrongSub_Context", 
                "WeakSub", "WeakSub_Context", "Polarity", "Positive", "Positive_Context", 
                "Negative", "Negative_Context", "Bias_Lexicon"]

from Neural_Network import*
Initialization(Task1Model_Path, Task2Model1_Path, Task2Model2_Path)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [9]:
## Testing

Test_Filepath = Folder_Path + "/Data/Model_Data/FinalTestData.csv"
Test_Data = pd.read_csv(Test_Filepath)
Test_Sentences = Test_Data['sentence']

Classification_labels = Bias_Detection_Suggestions(Test_Sentences, 20, 7)





For a few frequently occurring network motifs we show how they manage noise.
1/1 [==============================] - 0s 423us/step
Subjectivity not identified in the sentence



and in the dogs , she finds lost pieces of herself as she fumbles with the leash to her own life - struggling to hold on as it pulls her toward a brighter future before her past catches up with her .
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



Suggests two approaches to economics of blockchain: innovation-centred and governance-centred.
1/1 [==============================] - 0s 2ms/step
Subjectivity not identified in the sentence



being blue is not his only predicament ; he also lacks the ability to outwardly express his emotions .
1/1 [==============================] - 0s 954us/step
Subjectivity not identified in the sentence



on her deathbed , candice klein accidentally asks the question , " what did i ever do to deserve this ? "
1/1 [===============

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Subjectivity not identified in the sentence



It will also show how highly interlinked these disciplines are, and how developments in each drive and influence the other.
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



kok pin is creative and a born artist but his parents would rather he focus on his maths and sciences .
1/1 [==============================] - 0s 649us/step
Sentence is subjective
Word inducing subjectivity in the sentence:  creative
Alternate word suggetions: 
creativity
innovation
talent
marketing
experience
collaborative
talents



calvin ( cube ) , who inherited the struggling business from his deceased father , views the shop as nothing but a burden and waste of his time .
1/1 [==============================] - 0s 502us/step
Subjectivity not identified in the sentence



This paper examines how the changes in factors of a deep learning network can affect the output.
1/1 [==============================] - 0s 979us/ste

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Sentence is subjective
Word inducing subjectivity in the sentence:  unforgettable
Alternate word suggetions: 
evocation
enchanting
uplifting
haunting
heartbreaking
evocative
wondrous



against improbable odds , they race towards a thrilling , climactic confrontation on the mysterious isla de muerta .
1/1 [==============================] - 0s 429us/step
Sentence is subjective
Word inducing subjectivity in the sentence:  mysterious


...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Alternate word suggetions: 
guy
thing
look



Senior NFL Writer Jay Glazer is one of NFL's top insiders.
1/1 [==============================] - 0s 512us/step
Subjectivity not identified in the sentence



yet this highly traditional world is facing profound changes due to the postwar influx of foreign images and ideas in japanese culture .
1/1 [==============================] - 0s 844us/step
Subjectivity not identified in the sentence



she wants his blessing to marry a man she loves , but he tells her that she must marry a man of his choosing , and that she must undergo female circumcision .
1/1 [==============================] - 0s 404us/step
Subjectivity not identified in the sentence



for every inspired bit of absurdity , there's a lame piece of improvisational noodling .
1/1 [==============================] - 0s 1ms/step
Sentence is subjective
Word inducing subjectivity in the sentence:  lame
Alternate word suggetions: 
duck
joke
cranky



rayne johnson is a shrewd investment c

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Alternate word suggetions: 
way
think
thing



In this paper, we propose a new, fermionic Group Field Theory, posessing a color symmetry, and take the first steps in a systematic study of the topological properties of its graphs.
1/1 [==============================] - 0s 418us/step
Subjectivity not identified in the sentence



rooney in the past is sent to kill sullivan .
1/1 [==============================] - 0s 570us/step
Subjectivity not identified in the sentence



shot on location in the williamsburgh and flatbush neighborhoods of brooklyn , hung-up is a 35mm short story in the tradition of happy gilmore and taxi driver .
1/1 [==============================] - 0s 618us/step
Subjectivity not identified in the sentence



she entranced the founder of the museum of art ( masp ) where the fashion parade was being held , and her career was hence launched .
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



watching queen of the damned is

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [ ]:
# ## Testing

# Test_Filepath = Folder_Path + "/Data/Model_Data/test_sentences.csv"
# Test_Data = pd.read_csv(Test_Filepath)

# Test_Filtered_Data = []
# for i in range(Test_Data.shape[0]):
#     row = Test_Data.iloc[i]
    
#     if len(row['sentence'])>10:
#         Test_Filtered_Data.append(row)

# Test_Filtered_Data = pd.DataFrame(Test_Filtered_Data, columns=['sentence', 'label'])

# Test_Sentences = Test_Filtered_Data['sentence']
# Test_Labels = Test_Filtered_Data['label']


# # Test_Sentences = [
# #                   "nicaea was convoked by the emperor constantine in mayaugust 325 to address the arian heresy that christ is of a distinct substance from the father.",
# #                   "dennis the menace is an american animated series produced by dic entertainment, based on the classic comic strip by hank ketcham.",
# #                   "speculative strategies of life extension.",
# #                   "later editions of the work were famously illustrated by the norwegian artists erik werenskiold, theodor kittelsen, otto sinding and others.",
# #                   "blogger supports google's adsense service as a simple way of generating revenue from running a blog.",
# #                   "she was married to emil beaulieau, america's greatest living noisician.",
# #                   "what's worse is that pelosi knows it.",
# #                   "For a few frequently occurring network motifs we show how they manage noise."
# #                  ]

# Classification_labels = Bias_Detection_Suggestions(Test_Sentences)


# print("Test_Filtered_Data: ", Test_Filtered_Data.shape)

# Platinum_Dataset = []

# for i in range(Test_Filtered_Data.shape[0]):
#     Actual_label = Test_Labels.iloc[i]
#     Predicted_label = Classification_labels[i]
    
#     if Actual_label == Predicted_label:
#         sent = Test_Filtered_Data.iloc[i]
#         Platinum_Dataset.append(sent)
        
# Platinum_Dataset = pd.DataFrame(Platinum_Dataset, columns=['sentence', 'label'])
# Platinum_Dataset.to_csv(Folder_Path + "/Data/Model_Data/FinalTestData.csv")

# print("Platinum_Dataset: ", Platinum_Dataset.shape)

In [10]:
Classification_labels0 = [l for l in Classification_labels if l==0]
print("Objective: ", len(Classification_labels0))
print("Subjective: ", len(Classification_labels)-len(Classification_labels0))

Objective:  1586
Subjective:  658
